# One-dimensional deconvolution problem
Find $f(t), 0 \le t \le 1$ from noisy observations of the function
$$
g(s) = \int_{0}^{1} \phi(s-t) f(t) dt, 0 \le s \le 1,
$$
where the convolution kernel is
$$
\phi(t) = e^{-a|t|}, a = 20.
$$
As a test function, we use
$$
f(t) = t(1-t).
$$
Analytical solution of function $g$ yields
$$
g(s) = \frac{2}{a}s(1-s) + \frac{1}{a^2}(e^{-as} + e^{-a(1-s)}) + \frac{2}{a^3}(e^{-as} + e^{-a(1-s)} - 2).
$$
The data is recorded on an even mesh with mesh size 1/100. The reconstruction mesh is also equispaced with mesh size 1/80. Therefore, the matrix $A$ has entries
$$
A_{ij} = \frac{1}{80} e^{-a|t_i - s_j|}, t_i = \frac{i}{80}, s_j = \frac{j}{100},
$$
$$
0 \le i \le 80, 0 \le j \le 100.
$$
Random normally distributed zero mean noise is added to the exact data with independent components and standard deviation $5\%$ of the maximum value of the noiseless data.

In [1]:
%matplotlib qt
import math
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Given data
a = 20
t = [u / 80 for u in np.arange(0, 81)]
s = [v / 100 for v in np.arange(0, 101)]
g = [2 / a * sj * (1 - sj) + 1 / a ** 2 * (math.exp(-a * sj) + math.exp(-a * (1 - sj))) \
    + 2 / a ** 3 * (math.exp(-a * sj) + math.exp(-a * (1 - sj)) - 2) for sj in s]
f = [tj * (1 - tj) for tj in t]

In [3]:
# Generate the forward model matrix A
m, n = len(s), len(t)
A = np.zeros((m, n))
for i, ti in enumerate(t):
    for j, sj in enumerate(s):
        A[j][i] = 1 / 80 * math.exp(-a * np.abs(ti - sj))
print('Shape of A =', np.shape(A))

Shape of A = (101, 81)


In [4]:
# Generate noisy measurement data
inf_norm = np.max(np.abs(g))
sigma = 0.05 * inf_norm
eta = np.random.normal(scale=sigma, size=m)
y = (g + eta).reshape(-1, 1)

In [5]:
# Minimum norm solution
A_dag = np.linalg.pinv(A)
f_dag = A_dag @ y

In [6]:
# Figure 2.8
fig1 = plt.figure(figsize=(12, 6))
plt.suptitle('Figure 2.8')
ax11 = fig1.add_subplot(121)
plt.plot(s, y, lw=2, color='k', label='noisy')
plt.plot(s, g, lw=1, color='k', label='noiseless')
plt.xlabel('$s_j$', fontsize=14)
plt.ylabel('$g$', fontsize=14)
plt.legend()
ax21 = fig1.add_subplot(122)
plt.plot(t, f_dag, lw=1, color='k', label='minimum norm')
plt.plot(t, f, lw=2, color='k', label='true')
plt.xlabel('$t_i$', fontsize=14)
plt.ylabel('$f$', fontsize=14)
plt.legend()
plt.show()

# Solving using Landweber-Fridman iteration
The relaxation parameter was chosen as $\beta = 0.1\beta_{max}$, where $\beta_{max} = 2 / \lVert A \rVert ^2$. The error for the application of discrepancy principle was chosen as $\epsilon = \sqrt{81}\sigma$.

In [7]:
def regressLandweberFridman(A, b, x_0, i_max=1e3, epsilon=1e-6):
    '''
    Solve inverse problem using Landweber-Fridman iteration scheme
    '''
    beta_max = 2 / np.linalg.norm(A, ord=2) ** 2; beta = 0.1 * beta_max
    i, x = 0, x_0
    delta = np.linalg.norm(b - A @ x)
    X = []
    while (i < i_max) & (delta > epsilon):
        x = x + beta * (A.T @ b - A.T @ (A @ x))
        delta = np.linalg.norm(b - A @ x)
        X.append(x)
        i += 1
    return x, np.array(X), i

In [8]:
x_0 = np.zeros((np.size(A, 1), 1))
epsilon = math.sqrt(len(y)) * sigma # Morozov discrepancy principle
xLF, X, n_steps = regressLandweberFridman(A, y, x_0, epsilon=epsilon)
print('No. of iterations =', n_steps)

No. of iterations = 17


In [9]:
fig2 = plt.figure()
plt.suptitle('Figure 2.9')
plt.plot(t, f, color='k', lw=2, ls='--')
for x in X:
    plt.plot(t, x, color='k', lw=2)
plt.xlabel('$t_i$', fontsize=14)
plt.ylabel('$f_k$', fontsize=14)
plt.title('Number of iterations = %d' % n_steps)
plt.show()